## Advanced Lane Finding Project

The goals / steps of this project are the following:

* Compute the camera calibration matrix and distortion coefficients given a set of chessboard images.
* Apply a distortion correction to raw images.
* Use color transforms, gradients, etc., to create a thresholded binary image.
* Apply a perspective transform to rectify binary image ("birds-eye view").
* Detect lane pixels and fit to find the lane boundary.
* Determine the curvature of the lane and vehicle position with respect to center.
* Warp the detected lane boundaries back onto the original image.
* Output visual display of the lane boundaries and numerical estimation of lane curvature and vehicle position.

---
## First, I'll compute the camera calibration using chessboard images

# code1 : calibration

In [2]:
import numpy as np
import cv2
import glob
import matplotlib.pyplot as plt
from moviepy.editor import VideoFileClip
from IPython.display import HTML
%matplotlib qt
%matplotlib inline


def Calibration():
    # prepare object points, like (0,0,0), (1,0,0), (2,0,0) ....,(6,5,0)
    objp = np.zeros((6*9,3), np.float32)
    objp[:,:2] = np.mgrid[0:9,0:6].T.reshape(-1,2)

    # Arrays to store object points and image points from all the images.
    objpoints = [] # 3d points in real world space
    imgpoints = [] # 2d points in image plane.

    # Make a list of calibration images
    images = glob.glob('../camera_cal/calibration*.jpg')

    # Step through the list and search for chessboard corners
    for fname in images:
        img = cv2.imread(fname)
        gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)

        # Find the chessboard corners
        ret, corners = cv2.findChessboardCorners(gray, (9,6),None)

        # If found, add object points, image points
        if ret == True:
            objpoints.append(objp)
            imgpoints.append(corners)
            # Draw and display the corners
            cv2.drawChessboardCorners(img, (9,6), corners, ret)
            #plt.imshow(img)
            #cv2.waitKey(500) this was unnecessary 
    #calibration
    ret,mtx,dist,rvecs,trevs = cv2.calibrateCamera(objpoints,imgpoints,gray.shape[::-1],None,None)
    return mtx,dist,rvecs,trevs

In [3]:
def undistortImage(mtx,dist,img):
    # reduce distortions from an image and generate an undistorted image
    return cv2.undistort(img,mtx,dist,None,mtx) 

In [4]:
def birdView(undst):
    
    # define the xsize and ysize
    xsize = undst.shape[1]
    ysize = undst.shape[0]
    
    # crop an interested region
    # a grid in a front view
    offset_x_top = 50
    offset_y = 0.64
    offset_x_bottom = 200
    
    src = np.float32([[0.5*xsize-offset_x_top,ysize*offset_y],
                     [0.5*xsize+offset_x_top,ysize*offset_y],
                     [xsize-offset_x_bottom,1.0*ysize],
                     [offset_x_bottom,1.0*ysize]])
    
    
    # a grid in a bird view
    bird_undst = np.float32([[0.05*xsize,0.05*ysize],
                     [0.95*xsize,0.05*ysize],
                     [0.95*xsize,1.0*ysize],
                     [0.05*xsize,1.0*ysize]])
    # get a matrix of the linear trasnformation from src to bird_undst
    M = cv2.getPerspectiveTransform(src,bird_undst)
    # get an inverse matrix of the linear transformation from src to bird_undst
    M_r = cv2.getPerspectiveTransform(bird_undst,src)
    
    # Generate a bird view image from a front view image  
    warped = cv2.warpPerspective(undst,M,(xsize,ysize),flags=cv2.INTER_LINEAR)
    # Generate a front view image from a bird view image
    re_warped = cv2.warpPerspective(undst,M_r,(xsize,ysize),flags=cv2.INTER_LINEAR)
    
    return warped,re_warped

# code2 : color transformations

In [5]:
def hls_threshold(original_img,h_thresh=(50,100), s_thresh=(100,250)):
    
    # Convert to hls scale
    hls = cv2.cvtColor(original_img,cv2.COLOR_RGB2HLS)
    H = hls[:,:,0]
    L = hls[:,:,1]
    S = hls[:,:,2]
    
    # Create a binary image which has the same size as H
    binary_H = np.zeros_like(H)
    binary_H[(H > h_thresh[0]) & (H < h_thresh[1])] = 1
    
    # Create a binary image which has the same size as S
    binary_S = np.zeros_like(S)
    binary_S[(S > s_thresh[0]) & (S < s_thresh[1])] = 1
    
    return binary_H, binary_S

In [6]:
def sobelx_threshold(original_img,sobel_kernel=3,sobelx_thresh=(30,100)):
    # Convert to gray scale
    # gray is 2 dimentional
    """gray = cv2.cvtColor(original_img,cv2.COLOR_BGR2GRAY)"""
    gray = cv2.cvtColor(original_img,cv2.COLOR_RGB2GRAY)
    
    # apply sobelx and sobely to the gray image
    sobelx = cv2.Sobel(gray,cv2.CV_64F,1,0,ksize = sobel_kernel)
    
    abs_sobelx = np.absolute(sobelx)
    
    # firstly normalize the gradients, as the original abs_sobelxy can be more than 255
    # generate a 2 dimentional image with gradients ; a gray image
    scaled_sobel = np.uint8(255*abs_sobelx/np.max(abs_sobelx))

    # get minmum / maximum thresholds 
    thresh_min, thresh_max = sobelx_thresh
   
    # Create a binary image which has the same size as scaled_sobel
    sxbinary = np.zeros_like(scaled_sobel)
    sxbinary[(scaled_sobel > thresh_min)&(scaled_sobel < thresh_max)] = 1
    
    return sxbinary

In [7]:
def mag_threshold(original_img,sobel_kernel=3,mag_thresh=(30,100)):
    # Convert to gray scale
    # gray is 2 dimentional
    """gray = cv2.cvtColor(original_img,cv2.COLOR_BGR2GRAY)"""
    gray = cv2.cvtColor(original_img,cv2.COLOR_RGB2GRAY)
    
    # apply sobelx and sobely to the gray image
    sobelx = cv2.Sobel(gray,cv2.CV_64F,1,0,ksize = sobel_kernel)
    sobely = cv2.Sobel(gray,cv2.CV_64F,0,1,ksize = sobel_kernel)
    
    abs_sobelx = np.absolute(sobelx)
    abs_sobely = np.absolute(sobely)
    abs_sobelxy = np.sqrt(abs_sobelx **2 + abs_sobely**2)
    
    # firstly normalize the gradients, as the original abs_sobelxy can be more than 255
    # generate a 2 dimentional image with gradients ; a gray image
    scaled_sobel = np.uint8(255*abs_sobelxy/np.max(abs_sobelxy))

    # get minmum / maximum thresholds 
    thresh_min, thresh_max = mag_thresh
   
    # Create a binary image which has the same size as scaled_sobel
    magbinary = np.zeros_like(scaled_sobel)
    magbinary[(scaled_sobel > thresh_min)&(scaled_sobel < thresh_max)] = 1
    
    return magbinary

In [8]:
def dir_threshold(original_img,sobel_kernel=3,dir_thresh=(0,np.pi/2)):
    # Convert to gray scale
    # gray is 2 dimentional
    """gray = cv2.cvtColor(original_img,cv2.COLOR_BGR2GRAY)"""
    gray = cv2.cvtColor(original_img,cv2.COLOR_RGB2GRAY)
    
    # apply sobelx and sobely to the gray image
    sobelx = cv2.Sobel(gray,cv2.CV_64F,1,0,ksize = sobel_kernel)
    sobely = cv2.Sobel(gray,cv2.CV_64F,0,1,ksize = sobel_kernel)
    
    abs_sobelx = np.absolute(sobelx)
    abs_sobely = np.absolute(sobely)
    
    # take a theta 
    dir_grad = np.arctan2(abs_sobely,abs_sobelx)
    
    # get minmum / maximum thresholds 
    thresh_min ,thresh_max = dir_thresh
    
    # Create a binary image which has the same size as dir_grad
    sxbinary = np.zeros_like(dir_grad)
    sxbinary[(dir_grad > thresh_min)&(dir_grad < thresh_max)] = 1
    
    return sxbinary

# code 3 : Find lane pixels with windows 

In [9]:
def find_lane_pixels(binary_warped,nwindows= 9,margin =100,minpix= 50):
    
    # take the bottome half of the image 
    bottom_half = binary_warped[binary_warped.shape[0]//2:,:]
    
    # create a 3 dimentional image
    three_dim_warped = np.dstack((binary_warped,binary_warped,binary_warped))
    
    # make a histogram to decide 2 starting points on the left line and right line
    histogram = np.sum(bottom_half,axis = 0)
    
    # define a midpoint of the histogram
    midpoint = np.int(histogram.shape[0]//2)
    leftx_base = np.argmax(histogram[:midpoint])
    rightx_base = np.argmax(histogram[midpoint:]) + midpoint
    
    # define a height of a single window
    window_height = np.int(binary_warped.shape[0]//nwindows)
    
    # define a nonzero array which has x and y adresses of each none zero pixel
    nonzero = binary_warped.nonzero()
    nonzeroy = np.array(nonzero[0])
    nonzerox = np.array(nonzero[1])
    
    # current position of the 2 windows, one on the left and the other on the right
    leftx_current = leftx_base
    rightx_current = rightx_base
    
    # set empty arrays
    left_lane_inds = []
    right_lane_inds = []
    
    for window in range(nwindows):
        # define the size of the window
        win_y_low = binary_warped.shape[0] - (window + 1) * window_height
        win_y_high = binary_warped.shape[0] - window* window_height
        
        win_xleft_low = leftx_current - margin //2
        win_xleft_high = leftx_current + margin //2
        win_xright_low = rightx_current - margin //2
        win_xright_high = rightx_current + margin //2
        
        # On the left line-ish, assign the nonzero address(index) in the left window to good_left_ind
        good_left_inds = ((nonzeroy >= win_y_low)&
                          (nonzeroy <= win_y_high)&
                          (nonzerox >= win_xleft_low)&
                          (nonzerox <= win_xleft_high)).nonzero()[0]
        # so does for the right line-ish
        good_right_inds = ((nonzeroy >= win_y_low)&
                          (nonzeroy <= win_y_high)&
                          (nonzerox >= win_xright_low)&
                          (nonzerox <= win_xright_high)).nonzero()[0]
        
        # By assigning all the addresses from all the nwindows, left_lane_inds has all the x addresses of the left line- ish
        # so does the right line - ish
        left_lane_inds.append(good_left_inds)
        right_lane_inds.append(good_right_inds)
        
        # renew the left and right current postions of 2 windows 
        if len(good_left_inds) > minpix:
            left_current = np.int(np.mean(nonzerox[good_left_inds]))
        if len(good_right_inds) > minpix:
            rightx_current = np.int(np.mean(nonzerox[good_right_inds]))
            
         
    try:
        # turning left_lane_inds as a list to left_lane_inds as a ndarray
        left_lane_inds = np.concatenate(left_lane_inds)
        # and same for right
        right_lane_inds = np.concatenate(right_lane_inds)
        
    except ValueError:
        
        print("you shouldnt see this message")

    # output the x and y coordinates for the left and right line-ish pixel
    leftx = nonzerox[left_lane_inds]
    lefty = nonzeroy[left_lane_inds]
    rightx = nonzerox[right_lane_inds]
    righty = nonzeroy[right_lane_inds]
    
    return leftx,lefty,rightx,righty,three_dim_warped

In [10]:
def fit_polynomial(binary_warped):
    
    # get left and right line-ish addresses for x and y
    leftx,lefty,rightx,righty,three_dim_warped = find_lane_pixels(binary_warped,minpix=50)
    
    # generate a set of the coefficients out of the addresses 
    left_fit = np.polyfit(lefty,leftx,2)
    right_fit = np.polyfit(righty,rightx,2)
    
    # y-value from 0 to 719 (the tip of the image) 
    ploty = np.linspace(0,binary_warped.shape[0]-1,binary_warped.shape[0])
    
    # generate a left and right line out of the coefficients
    left_fitx = left_fit[0] * ploty**2 + left_fit[1] * ploty + left_fit[2]
    right_fitx = right_fit[0] * ploty**2 + right_fit[1] * ploty + right_fit[2]

    # Paint the line-ish pixels with red for the left, and blue for the right
    three_dim_warped[lefty,leftx] = [255,0,0]
    three_dim_warped[righty,rightx] = [0,0,255]
    
    # the three_dim_warped is colored with red and blue
    return three_dim_warped, left_fit, right_fit, left_fitx, right_fitx, ploty

In [11]:
def search_around_poly(binary_warped,left_fit,right_fit):
    margin = 100

    # Grab activated pixels
    nonzero = binary_warped.nonzero()
    nonzeroy = np.array(nonzero[0])
    nonzerox = np.array(nonzero[1])
    
    # output a boolean array if the element of the binary picture is in the range with the elements of the fit line with margin  
    left_lane_inds = ((nonzerox > (left_fit[0]*(nonzeroy**2) + left_fit[1]*nonzeroy + 
                    left_fit[2] - margin)) & (nonzerox < (left_fit[0]*(nonzeroy**2) + 
                    left_fit[1]*nonzeroy + left_fit[2] + margin)))
    right_lane_inds = ((nonzerox > (right_fit[0]*(nonzeroy**2) + right_fit[1]*nonzeroy + 
                    right_fit[2] - margin)) & (nonzerox < (right_fit[0]*(nonzeroy**2) + 
                    right_fit[1]*nonzeroy + right_fit[2] + margin)))
    
    # Again, extract left and right line pixel positions
    leftx = nonzerox[left_lane_inds]
    lefty = nonzeroy[left_lane_inds] 
    rightx = nonzerox[right_lane_inds]
    righty = nonzeroy[right_lane_inds]

    # get coefficients of the right and left line
    left_fit = np.polyfit(lefty,leftx,2)
    right_fit = np.polyfit(righty,rightx,2)
    
    # an array of y element's values
    ploty = np.linspace(0,binary_warped.shape[0]-1,binary_warped.shape[0])
    # Fit two new polynomials
    left_fitx = left_fit[0] * ploty**2 + left_fit[1] * ploty + left_fit[2]
    right_fitx = right_fit[0] * ploty**2 + right_fit[1] * ploty + right_fit[2]
    
    # return the new coefficients of the 2 new polynomials
    return left_fit, right_fit, left_fitx, right_fitx , ploty

In [12]:
def Visualization(three_dim_warped,left_fitx, right_fitx , ploty):

    # the argument image is 3 dimentional to paint the drivable space on

    # an array with x and y coordinates of left lane line from top to bottom 
    left_line = np.array([np.transpose(np.vstack([left_fitx,ploty]))])
    
    # an array with x and y coordinates of right lane line from bottom to top
    right_line = np.array([np.flipud(np.transpose(np.vstack([right_fitx,ploty])))])
    
    # make a 2 layer of x and y 
    line_pts = np.hstack((left_line,right_line))
  
    # prepare a black image to paint the drivable space on
    window_img = np.zeros_like(three_dim_warped)
    
    # paint green on the image above
    cv2.fillPoly(window_img, np.int_([line_pts]), (0,255, 0))
    
    # transform the bird-viewed image to a front view image
    warped, re_warped = birdView(window_img)
    
    # return a front view image
    return re_warped

In [13]:
def stackimages(oringinal_img,lane_img):
    # stack an colored lane image on an original image
    return cv2.addWeighted(oringinal_img,1,lane_img,0.5,0)

In [14]:
def caluculate_Curve(left_fit, right_fit, ploty):
    # adjust a scale from pixel unit to a meter unit
    xm_scale = 3.7/700
    ym_scale = 30/720
    
    # get a maximum value of y in meter unit
    max_y = np.max(ploty) * ym_scale
    
    # the carvetures of the left line and the right line
    Radius_left = ((1 + (2*left_fit[0]*max_y + left_fit[1])**2)**1.5) / abs(2*left_fit[0])
    Radius_right = ((1 + (2*right_fit[0]*max_y + right_fit[1])**2)**1.5) / abs(2*right_fit[0])
   
    
    # output the text with the value of the carveture
    text = "Radius is {}(m)".format(np.mean([Radius_left,Radius_right]) // 1 )
    
    return text

In [15]:
def vehicle_position(out_img,left_fitx, right_fitx , ploty):
    # adjust a scale from pixel unit to a meter unit
    xm_scale = 3.7/700
    ym_scale = 30/720
    
    # define a midpoint of the 2 lane lines
    lane_line_mean = np.mean((right_fitx[len(ploty)-1],left_fitx[len(ploty)-1]))
    # difference of the midpoint of the img and lane_line_mean 
    # if diff is positive then right, otherwise left
    diff = out_img.shape[1] // 2 - lane_line_mean
    
    if(diff > 0):
        text = "The vehicle is {:.2f} m right of center".format(diff*xm_scale)
    else:
        text = "The vehicle is {:.2f} m left of center".format(abs(diff*xm_scale))
    
    return text

In [16]:
#Output visual display of the lane boundaries and numerical estimation of lane curvature and vehicle position
def last_compile(frame):
   
    global flag_frame
    global left_fit
    global right_fit
    global mtx,dist
    original_img = np.copy(frame)
    
    # frame number 1
    if(flag_frame == False):
        # undistort img 
        undistorted_img = undistortImage(mtx,dist,original_img)
        
        # making binary images based on the bird-viewed pictures 
        h_binary, s_binary = hls_threshold(undistorted_img,h_thresh=(0,80), s_thresh=(90, 255))
        sx_binary = sobelx_threshold(undistorted_img, sobel_kernel=3, sobelx_thresh=(20,100))
        mag_binary = mag_threshold(undistorted_img,sobel_kernel=27,mag_thresh=(70,210))
        dir_binary = dir_threshold(undistorted_img,sobel_kernel=3,dir_thresh=(-np.pi/8,np.pi/4))
        combined = np.zeros_like(h_binary)
        combined[(sx_binary==1)|(s_binary == 1) | (mag_binary ==1)&(dir_binary ==1)] = 1

        # get a linear transformation and a reverse linear transformation image
        warped,re_warped = birdView(combined)

        # fit a polinomial out of the image and outout the coefficients,coordinates 
        # and the image with colored lane lines   
        three_dimentional_img, left_fit, right_fit, left_fitx, right_fitx, ploty = fit_polynomial(warped)
        flag_frame = True
    # frame number 2 and so on
    else:
        exp = undistortImage(mtx,dist,original_img)
        h_binary, s_binary = hls_threshold(exp,h_thresh=(0,80), s_thresh=(90, 255))
        sx_binary = sobelx_threshold(exp, sobel_kernel=3, sobelx_thresh=(20,100))
        mag_binary = mag_threshold(exp,sobel_kernel=27,mag_thresh=(70,210))
        dir_binary = dir_threshold(exp,sobel_kernel=3,dir_thresh=(-np.pi/8,np.pi/4))
        combined = np.zeros_like(h_binary)
        combined[(sx_binary==1)|(s_binary == 1) | (mag_binary ==1)&(dir_binary ==1)] = 1
        
        # get a linear transformation and a reverse linear transformation image
        warped,re_warped = birdView(combined)
    
        left_fit, right_fit, left_fitx, right_fitx , ploty = search_around_poly(warped,left_fit,right_fit)
        
    text_1 = caluculate_Curve(left_fit, right_fit, ploty) 
    text_2 = vehicle_position(original_img,left_fitx, right_fitx , ploty)
    vision =  Visualization(original_img,left_fitx, right_fitx, ploty)
    vision = stackimages(original_img,vision)
    cv2.putText(vision, text_1, (20,100),cv2.FONT_HERSHEY_SIMPLEX, 3, (255, 255, 255),10, cv2.LINE_AA)
    cv2.putText(vision, text_2, (20,200),cv2.FONT_HERSHEY_SIMPLEX, 2, (255, 255, 255),5, cv2.LINE_AA)
    
    return vision

In [17]:
flag_frame = False
left_fit, right_fit = [],[]
mtx,dist,rvecs,trevs = Calibration()

white_output = '../output_challenge_video.mp4'
clip1 = VideoFileClip("../challenge_video.mp4")
white_clip = clip1.fl_image(last_compile)
%time white_clip.write_videofile(white_output,audio=False)

[MoviePy] >>>> Building video ../output_challenge_video.mp4
[MoviePy] Writing video ../output_challenge_video.mp4


100%|██████████| 485/485 [03:02<00:00,  2.57it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: ../output_challenge_video.mp4 

CPU times: user 1min 56s, sys: 21.7 s, total: 2min 17s
Wall time: 3min 5s


In [18]:
HTML("""
<video width="960" height="540" controls>
<source src="{0}">
</video>
""".format(white_output))